## Getting Started

In [ ]:
#!pip install ipywidgets

In [ ]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate

from langchain import LLMChain

from langchain.tools import DuckDuckGoSearchRun 
from langchain.utilities import GoogleSearchAPIWrapper

from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re
import langchain
from togetherllm import TogetherLLM
import pinecone
from langchain.vectorstores import Pinecone
from langchain.chains import LLMMathChain
from langchain.prompts import PromptTemplate

from langchain.agents import Tool, AgentExecutor, initialize_agent


In [ ]:
llm = TogetherLLM(
    model= "togethercomputer/llama-2-7b-chat",
    temperature=0,
    max_tokens=512
)

## Let's create our Tool
- 1st Tool : To create summarizer tools
- 2nd Tool : To create Q&A tools
- 3rd Tool : To create LLM Math Chain tools
- 4th Tool : To create a general tools

In [ ]:
### Math 
llm_math = LLMMathChain.from_llm(llm=llm, verbose=True)

## Duckduckgosearch
duck = DuckDuckGoSearchRun()

tools = []

In [ ]:
custom_prompt_template = """[INST] <<SYS>>
Your name is Kelly, You are helpful assistant, you always open and only answer for the assistant then you stop.
If you don't know the answer, just say you don't know and submit the request to hotline@xfab.com for further assistance.
<</SYS>>

Question: {question}
[/INST]"""

In [ ]:
prompt = PromptTemplate(input_variables=['question'],template=custom_prompt_template)

In [ ]:
### Math 
llm_math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description='use this tool to perform calculations'
)

In [ ]:
### General Tools
llm_chain = LLMChain(llm=llm, prompt=prompt)
general_tool = Tool(
    name='Language Model',
    func=llm_chain.run,
    description="Useful tool when you need  general purpose queries and logic'"

)

In [ ]:
duck_tool = Tool(
        name="DuckDuckGo",
        func=duck.run,
        description="Useful for when you need to do a search on the internet to find information that another tool can't find. Be specific with your input."
    )

In [ ]:
tools.extend([llm_math_tool, general_tool, duck_tool])

In [ ]:
# Using Zero Shot
agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    #verbose=True,
    max_iterations=3,
    handle_parsing_errors="Check your output and make sure it conforms!",
    early_stopping_method="generate", #to perform one FINAL pass through the LLM to generate an output
)

In [ ]:
agent("Write me python code that convert excel to csv ?")

In [ ]:
try:
    response= agent.run("Write me python code that convert excel to csv ?")
    print(response)
except Exception as e:
        response = str(e)
        if response.startswith("Could not parse LLM output: `"):
            response = response.removeprefix("Could not parse LLM output: `").removesuffix("`")

## Agent Executor
This instructs the agent on what to do. Generally, the template should incorporate:

tools: which tools the agent has access and how and when to call them.

intermediate_steps: These are tuples of previous (AgentAction, Observation) pairs. These are generally not passed directly to the model, but the prompt template formats them in a specific way.

input: generic user input


In [ ]:
# Set up the base template
template_with_history = """Answer the following questions as best you can, but speaking as helpful customer support assistant. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to answer as a as helpful customer support assistant when giving your final answer.

Previous conversation history:
{history}

New question: {input}
{agent_scratchpad}"""

In [ ]:
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [ ]:
prompt_with_history = CustomPromptTemplate(
    template=template_with_history,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps", "history"]
)

## Custom Output Parser
The output parser is responsible for parsing the LLM output into AgentAction and AgentFinish. This usually depends heavily on the prompt used.

This is where you can change the parsing to do retries, handle whitespace, etc

In [ ]:
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [ ]:
output_parser = CustomOutputParser()

## Set Up LLM
Choose the LLM you want to use!

In [ ]:
llm = TogetherLLM(
model= "togethercomputer/llama-2-7b-chat",
temperature=1,
max_tokens=512
)

## Define the stop sequence
This is important because it tells the LLM when to stop generation.

This depends heavily on the prompt and model you are using. Generally, you want this to be whatever token you use in the prompt to denote the start of an Observation (otherwise, the LLM may hallucinate an observation for you).

## Set up the Agent
We can now combine everything to set up our agent

In [ ]:
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt_with_history)

In [ ]:
tool_names = [tool.name for tool in tools]
print(tool_names)
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

## What is an Agent Executor?
Agent Executors take an agent and tools and use the agent to decide which tools to call and in what order.

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory=ConversationBufferWindowMemory(k=4)

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, 
    tools=tools, 
    verbose=True, 
    memory=memory
    )

In [ ]:
agent_executor.run("Please Write me a formal letter to school for drone budget request for S.T.EM") 

In [ ]:
agent_executor.run("Find value of x for this equation 2x+5=6+x") 